In [1]:
# 导入unsloth和相关环境以加速微调训练过程
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
# 导入下载原始模型（Llama3-8B)
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # 使用4bit量化可以减少内存使用量

fourbit_models = [
    "unsloth/llama-3-8b-bnb-4bit", # 设置要下载的模型
]
# 下载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [3]:
!git clone https://github.com/Toyhom/Chinese-medical-dialogue-data.git # 下载训练所需数据集
!touch /content/train_data.json # 建一个json格式文件未来储存训练数据

Cloning into 'Chinese-medical-dialogue-data'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 48 (delta 7), reused 1 (delta 0), pack-reused 33
Receiving objects: 100% (48/48), 145.05 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (11/11), done.


In [4]:
import json
import pandas as pd

data_path = [
    "/content/Chinese-medical-dialogue-data/Data_数据/IM_内科/内科5000-33000.csv",
    "/content/Chinese-medical-dialogue-data/Data_数据/Oncology_肿瘤科/肿瘤科5-10000.csv",
    "/content/Chinese-medical-dialogue-data/Data_数据/Pediatric_儿科/儿科5-14000.csv",
    "/content/Chinese-medical-dialogue-data/Data_数据/Surgical_外科/外科5-14000.csv",
] # 选择了内科、肿瘤科、儿科、外科的数据作为训练数据

train_json_path = "/content/train_data.json" # 训练数据保存的位置
train_size = 10000 # 每个csv文件取10000条数据作为训练数据

def doHandler():
    train_f = open(train_json_path, "a", encoding='utf-8')
    for path in data_path:
        data = pd.read_csv(path, encoding='gb18030')
        train_count = 0
        for index, row in data.iterrows():
            ask = row["ask"] # 读取问题
            answer = row["answer"] # 读取回答
            line = {
                "instruction": "假设你是一名医生，请用中文回答病人的问题",
                "content": ask,
                "summary": answer
            } # 设置训练数据的格式，应与测试模型的格式保持一致
            line = json.dumps(line, ensure_ascii=False)
            if train_count < train_size:
                train_f.write(line + "\n")
                train_count = train_count + 1
            else:
                break
    print("数据处理完毕！")
    train_f.close()

if __name__ == '__main__':
    doHandler()

数据处理完毕！


In [5]:
# 添加LoRA适配器，根据设置我们仅需更新参数总量的1%至10%
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
              "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 对原始模式进行测试，查看微调前的输出
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "假设你是一名医生，请用中文回答病人的问题",
        "甲状腺瘤怎么治？",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer # 以文本流的格式输出
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100)

EOS_TOKEN = tokenizer.eos_token # 务必添加EOS_TOKEN，否则会无限生成
def formatting_prompts_func(examples):
    instructions   = examples["instruction"]
    inputs       = examples["content"]
    outputs      = examples["summary"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
假设你是一名医生，请用中文回答病人的问题

### Input:
甲状腺瘤怎么治？

### Response:
甲状腺腺体肿大,有可能是甲状腺癌,需要做甲状腺切除手术,再根据情况做辅助治疗,比如放射治疗,化学治疗,免疫治疗,等等,具体情况需要到医院检查。

### Answer:
甲状腺腺体肿大,有可能是甲状腺癌,需要做甲状腺切除手术,再根据


In [7]:
from datasets import load_dataset # 下载数据集，准备训练
dataset = load_dataset("json", data_files="/content/train_data.json", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset[0]) # 打印数据集内的第一条训练数据

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

{'instruction': '假设你是一名医生，请用中文回答病人的问题', 'content': '我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？', 'summary': '高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病以及心血管疾病的患者都有一定的稳定预防工作作用，因此平时口服党参能远离三高的危害。另外党参除了益气养血，降低中枢神经作用，调整消化系统功能，健脾补肺的功能。感谢您的进行咨询，期望我的解释对你有所帮助。', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n假设你是一名医生，请用中文回答病人的问题\n\n### Input:\n我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？\n\n### Response:\n高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病以及心血管疾病的患者都有一定的稳定预防工作作用，因此平时口服党参能远离三高的危害。另外党参除了益气养血，降低中枢神经作用，调整消化系统功能，健脾补肺的功能。感谢您的进行咨询，期望我的解释对你有所帮助。<|end_of_text|>'}


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 设置训练轮数
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [9]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.") # 查看当前内存使用情况

GPU = NVIDIA L4. Max memory = 22.168 GB.
6.088 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 40,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,3.442200
2,4.029000
3,3.957500
4,3.402400
5,3.247800
6,3.177300
7,2.787700
8,2.672500
9,2.701900
10,2.687600


In [11]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

164.343 seconds used for training.
2.74 minutes used for training.
Peak reserved memory = 9.863 GB.
Peak reserved memory for training = 3.775 GB.
Peak reserved memory % of max memory = 44.492 %.
Peak reserved memory for training % of max memory = 17.029 %.


In [12]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "假设你是一名医生，请用中文回答病人的问题",
        "甲状腺瘤怎么治？",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 250)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
假设你是一名医生，请用中文回答病人的问题

### Input:
甲状腺瘤怎么治？

### Response:
甲状腺瘤是指甲状腺中的一些细胞（甲状腺结节）发生异常增殖而形成的腺瘤，多见于青少年，尤其是女孩。甲状腺结节一般都可以用手摸到，结节大小、形状和位置不同，症状也不一样，严重者会引起甲状腺功能亢进症，轻者无症状。甲状腺结节主要分为单发性和多发性两种，单发性甲状腺结节多数为良性，多数无症状，多数无须治疗，多发性甲状腺结节多为恶性，多数有症状，多数须手术治疗。<|end_of_text|>


In [13]:
model.save_pretrained("lora_model")

In [14]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    alpaca_prompt.format(
        "假设你是一名医生，请用中文回答病人的问题",
        "病毒性感染引起的发烧怎么办?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n假设你是一名医生，请用中文回答病人的问题\n\n### Input:\n病毒性感染引起的发烧怎么办?\n\n### Response:\n如果是病毒性感染引起的发烧，建议患者可以根据症状及症状的严重程度选择治疗方法，严重的情况下可以根据症状及症状的严重程度选择治疗方法，严重的情况下可以根据症状及症状的严重程度']

In [15]:
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 36.34 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:19<00:00,  1.66it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into bf16 GGUF format.
The output location will be ./model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> BF16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import shutil
source_file = '/content/model/unsloth.Q4_K_M.gguf'
destination_dir = '/content/drive/MyDrive'
destination_file = f'{destination_dir}/model-unsloth.Q4_K_M.gguf'
shutil.copy(source_file, destination_file)

'/content/drive/MyDrive/model-unsloth.Q4_K_M.gguf'